In [0]:
train_df = spark.read.load("/Volumes/dai/phase2/silver/train_df")
test_df = spark.read.load("/Volumes/dai/phase2/silver/test_df")

In [0]:
from pyspark.sql.functions import rand

combined_df = train_df.unionByName(test_df)
combined_df = combined_df.orderBy(rand())
display(combined_df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight,features
17624,843.4799999999997,50,554,2011-11-25,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""843.4799999999997"",""50.0"",""554.0""]}"
16217,259.39000000000004,19,76,2011-04-18,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""259.39000000000004"",""19.0"",""76.0""]}"
14709,4911.140000000007,197,2482,2011-11-24,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""4911.140000000007"",""197.0"",""2482.0""]}"
17323,908.9900000000005,204,770,2011-11-24,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""908.9900000000005"",""204.0"",""770.0""]}"
15821,570.96,62,202,2011-10-26,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""570.96"",""62.0"",""202.0""]}"
15327,208.75000000000003,10,116,2011-10-18,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""208.75000000000003"",""10.0"",""116.0""]}"
14756,738.9599999999998,38,416,2011-11-24,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""738.9599999999998"",""38.0"",""416.0""]}"
17932,1331.5699999999997,90,789,2011-11-10,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1331.5699999999997"",""90.0"",""789.0""]}"
17251,283.28,39,110,2010-12-12,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""283.28"",""39.0"",""110.0""]}"
14959,684.9699999999999,61,491,2011-10-11,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""684.9699999999999"",""61.0"",""491.0""]}"


In [0]:
# combined_df.write.format("delta").mode("overwrite").save("/Volumes/dai/phase2/silver/data_for_training_or_prediction")

In [0]:
df = spark.read.load("/Volumes/dai/phase2/silver/data_for_training_or_prediction")
display(df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight,features
15937,145.35,7,73,2011-10-06,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""145.35"",""7.0"",""73.0""]}"
17669,3406.7200000000003,68,1916,2011-10-28,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""3406.7200000000003"",""68.0"",""1916.0""]}"
13258,698.5799999999999,38,287,2011-11-28,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""698.5799999999999"",""38.0"",""287.0""]}"
16810,275.64,16,112,2011-05-31,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""275.64"",""16.0"",""112.0""]}"
14273,559.9700000000001,29,264,2011-10-19,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""559.9700000000001"",""29.0"",""264.0""]}"
18205,517.36,32,192,2011-11-03,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""517.36"",""32.0"",""192.0""]}"
13494,316.26,18,162,2011-02-01,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""316.26"",""18.0"",""162.0""]}"
14142,311.81000000000006,22,313,2010-12-01,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""311.81000000000006"",""22.0"",""313.0""]}"
16323,207.50000000000003,1,50,2011-05-27,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""207.50000000000003"",""1.0"",""50.0""]}"
14862,841.2000000000002,48,772,2011-11-14,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""841.2000000000002"",""48.0"",""772.0""]}"


### Loading the model

In [0]:
import mlflow

In [0]:
model = mlflow.spark.load_model(
    "models:/dai.phase2.rf_model_for_ecommerce/1",
    dfs_tmpdir="/Volumes/dai/phase2/silver/tmp"
)

/databricks/python/lib/python3.12/site-packages/databricks/sdk/errors/base.py:87: UserWarning: The 'retry_after_secs' parameter of DatabricksError is deprecated and will be removed in a future version.
  warnings.warn(


In [0]:
predictions = model.transform(df)

In [0]:
display(predictions.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight,features,rawPrediction,probability,prediction
15937,145.35,7,73,2011-10-06,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""145.35"",""7.0"",""73.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
17669,3406.7200000000003,68,1916,2011-10-28,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""3406.7200000000003"",""68.0"",""1916.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""24.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}",1.0
13258,698.5799999999999,38,287,2011-11-28,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""698.5799999999999"",""38.0"",""287.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
16810,275.64,16,112,2011-05-31,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""275.64"",""16.0"",""112.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
14273,559.9700000000001,29,264,2011-10-19,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""559.9700000000001"",""29.0"",""264.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
18205,517.36,32,192,2011-11-03,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""517.36"",""32.0"",""192.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
13494,316.26,18,162,2011-02-01,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""316.26"",""18.0"",""162.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
14142,311.81000000000006,22,313,2010-12-01,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""311.81000000000006"",""22.0"",""313.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
16323,207.50000000000003,1,50,2011-05-27,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""207.50000000000003"",""1.0"",""50.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0
14862,841.2000000000002,48,772,2011-11-14,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""841.2000000000002"",""48.0"",""772.0""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""23.998626972218453"",""0.0013730277815488657""]}","{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}",0.0


In [0]:
predictions.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- total_spent: double (nullable = true)
 |-- total_transactions: long (nullable = true)
 |-- total_quantity: long (nullable = true)
 |-- last_purchase_date: date (nullable = true)
 |-- is_high_valued: integer (nullable = true)
 |-- class_weight: double (nullable = true)
 |-- features: vectorudt (nullable = true)
 |-- rawPrediction: vectorudt (nullable = true)
 |-- probability: vectorudt (nullable = true)
 |-- prediction: double (nullable = false)



## Creating a `gold` volume and storing these predictions 

In [0]:
# spark.sql("""
#           create volume if not exists dai.phase2.gold
# """)

DataFrame[]

In [0]:
predictions_df = predictions.select("CustomerID", "total_spent", 'total_transactions', 'total_quantity', 'last_purchase_date','is_high_valued' , "prediction", "probability")

display(predictions_df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,prediction,probability
15937,145.35,7,73,2011-10-06,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
17669,3406.7200000000003,68,1916,2011-10-28,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}"
13258,698.5799999999999,38,287,2011-11-28,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
16810,275.64,16,112,2011-05-31,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
14273,559.9700000000001,29,264,2011-10-19,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
18205,517.36,32,192,2011-11-03,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
13494,316.26,18,162,2011-02-01,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
14142,311.81000000000006,22,313,2010-12-01,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
16323,207.50000000000003,1,50,2011-05-27,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"
14862,841.2000000000002,48,772,2011-11-14,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.9999427905091021"",""5.72094908978694E-5""]}"


In [0]:
predictions_df.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/dai/phase2/gold/gold_predctions")

In [0]:
predictions.write.format("delta") \
    .mode("overwrite") \
    .save("/Volumes/dai/phase2/gold/predictions_raw")